### Push motion of `Panda` and contact detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco_py
from mujoco_parser import *
from util import *
np.set_printoptions(precision=2)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("Done.")

### Parse `Panda` with obejcts

In [ ]:
env = MuJoCoParserClass(name='Panda',rel_xml_path='../asset/panda/franka_panda_w_objs.xml',VERBOSE=True)
put_env_objets_in_a_row(env,prefix='obj_',x_obj=-1.0,z_obj=0.0)
env.init_viewer(TERMINATE_GLFW=False,INITIALIZE_GLFW=True,window_width=0.5,window_height=0.5)
env.plot_scene(figsize=(10,6),render_w=1000,render_h=600,
               title_str='Initial scene',title_fs=15,RETURN_IMG=False)

In [ ]:
# Simple render
env.reset(RESET_SIM=True)
put_env_objets_in_a_row(env,prefix='obj_',x_obj=-1.0,z_obj=0.0)
env.init_viewer(TERMINATE_GLFW=True,INITIALIZE_GLFW=True,window_width=0.5,window_height=0.5)
env.set_max_sec(max_sec=60)

# Objects random spawn
n_place         = 10 # number of objects to place
x_range         = [0.3,1.0]
y_range         = [-0.5,0.5]
z_range         = [1.1,1.1]
put_env_objets_in_a_row(env,prefix='obj_',x_obj=-1.0,z_obj=0.0) # init objects
object_names    = get_env_object_names(env,prefix='obj_')
n_object        = len(object_names)
object_idxs     = np.random.permutation(n_object)[:n_place].astype(int)
object_pos_list = np.zeros((n_place,3))
for o_idx in range(n_place):
    while True:
        x = np.random.uniform(low=x_range[0],high=x_range[1])
        y = np.random.uniform(low=y_range[0],high=y_range[1])
        z = np.random.uniform(low=z_range[0],high=z_range[1])
        xyz = np.array([x,y,z])
        if o_idx >= 1:
            devc = cdist(xyz.reshape((-1,3)),object_pos_list[:o_idx,:].reshape((-1,3)),'euclidean')
            if devc.min() > 0.15: break # minimum distance between objects
        else:
            break
    object_pos_list[o_idx,:] = xyz
object_placed_names = [object_names[object_idx] for object_idx in object_idxs]
set_env_objects(env,object_names=object_placed_names,object_pos_list=object_pos_list,colors=None)

while env.IS_ALIVE():
    env.step()
    marker_xyzs = np.array([[0.3,0,1],[0.3,0.5,1],[0.3,-0.5,1],[1,0,1],[1,0.5,1],[1,-0.5,1],
                            [0.3,0,1.2],[0.3,0.5,1.2],[0.3,-0.5,1.2],[1,0,1.2],[1,0.5,1.2],[1,-0.5,1.2]])
    # env.add_markers(pos_list=marker_xyzs,radius=0.02,color=np.array([0,0,1,1]))
    
    # Check contacts
    for c_idx in range(env.sim.data.ncon):
        contact    = env.sim.data.contact[c_idx]
        # Compute contact point and force
        p_contact  = contact.pos
        f_contact = np.zeros(6,dtype=np.float64) 
        mujoco_py.functions.mj_contactForce(env.sim.model,env.sim.data,c_idx,f_contact)
        # The contact force is in the contact frame
        contact_frame = contact.frame
        R_frame       = contact_frame.reshape((3,3))
        f_contact_global = R_frame @ f_contact[:3]
        f_norm = np.linalg.norm(f_contact_global)
        
        # Contacting bodies
        bodyid1    = env.sim.model.geom_bodyid[contact.geom1]
        bodyid2    = env.sim.model.geom_bodyid[contact.geom2]
        bodyname1  = env.body_idx2name(bodyid1)
        bodyname2  = env.body_idx2name(bodyid2)
        label      = ''# '%s-%s'%(bodyname1,bodyname2)
        # Plot contact force
        # env.add_marker(pos=p_contact,radius=0.03,color=np.array([1,0,0,0.5]),label=label)
        env.add_arrow(pos=p_contact,uv_arrow=f_contact_global,r_stem=0.02,len_arrow=f_norm/20,
                      color=np.array([1, 0, 0.5, 0.3]),label='')
        env.add_arrow(pos=p_contact,uv_arrow=-f_contact_global,r_stem=0.02,len_arrow=f_norm/20,
                      color=np.array([0.5, 0, 1, 0.3]),label='')
        
    env.render()
env.terminate_viewer()
print ("Done.")